In [1]:
import numpy as np
import scipy.io as sio
import numpy.matlib as npm
from scipy import signal
import pandas as pd
from numpy import linalg as LA
from scipy import linalg
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from pynq import Overlay
from pynq import allocate

In [2]:
nop=200
dim_r = 1024
Nu_elements = 32 # Variable Number of Antenna
dim_c = Nu_elements
angles = 181
buffer_size = 2*dim_r*dim_c 
y_buffer_size = 2*angles*dim_r
angle_prec=1

In [3]:
class radar1:
    def __init__(self):
        self.fc = 60*(10**9)
        self.fs = 1.76*(10**9)
        self.L = 512
        self.PRI = 2*(10**-6)
        self.c = 3*(10**8)
        self.ts = 1/self.fs
        self.PTM_len = nop
        self.Lambda = self.c/self.fc
        self.dfd = 121
        self.im_fdmax = 12*(10**3)
        self.im_fdmin = -12*(10**3)
        
class target:
    def __init__(self, range_t, doppler_velocity, rcs_amp):
        self.range = range_t
        self.doppler_velocity = doppler_velocity
        self.rcs_amp = rcs_amp
        self.n_bodypart = 1

class antenna:
    def __init__(self, Lambda, Nelements, DOA):
        self.d_m = Lambda * 0.5
        self.Nelements = Nelements
        self.spacing = np.arange(0, (self.Nelements) * self.d_m, self.d_m)
        self.DOA = DOA # Angle for azimuth

def get_received_signal(target, radar1, xmat1, fs, fc, antenna, n_frame):
    Mvec = radar1.PRI * np.arange(0,radar1.PTM_len)
    Marray = npm.repmat(Mvec, radar1.L, 1)
    radar1.ptx = 40
    radar1.gt = 0
    radar1.gr = 0
    rxmat = np.zeros((2*radar1.L, radar1.PTM_len), dtype=np.complex64)
    rxmat_temp = np.zeros((2*radar1.L-1, radar1.PTM_len), dtype=np.complex64)
    range_to_delay_symbols = np.floor(((2*target.range)/radar1.c)/radar1.ts)
    delay_zero = np.zeros((int(range_to_delay_symbols) - 1, int(radar1.PTM_len)))
    suffix_zero = np.zeros((int(radar1.L - range_to_delay_symbols), int(radar1.PTM_len)))
    
    # Range and doppler delay
    radar1.ptx_linear = (10**(radar1.ptx//10))*np.float32(0.001)
    radar1.gt_linear = (10**(radar1.gt//10))
    radar1.gr_linear = (10**(radar1.gr//10))
    radar1.radar_coeff=(np.sqrt(radar1.ptx_linear*radar1.gt_linear*radar1.gr_linear)*radar1.Lambda)/np.sqrt((4*np.pi)**3)   
    for nb in range(0,target.n_bodypart):
        velvec = target.doppler_velocity
        Doppler_ele = 2*np.pi*2*velvec/radar1.Lambda
        Doppler_Delay = np.exp(1j*(Doppler_ele*Marray))   
        temp = np.multiply(xmat1,Doppler_Delay)      
        amp = np.sqrt(target.rcs_amp)/(target.range ** 2)
        rxmat_temp = (amp*radar1.radar_coeff )*np.concatenate((delay_zero, temp, suffix_zero), dtype = np.complex64)
        rxmat[0:1023,:] = rxmat_temp

    # azimuth delay
    srx = np.zeros((2 * radar1.L, radar1.PTM_len, antenna.Nelements), dtype=np.complex64)
    lambda_m = radar1.Lambda
    antenna_matrix = np.exp(1j * 2 * np.pi * (1 / lambda_m) * np.sin(np.deg2rad(antenna.DOA)) * antenna.spacing)
    for p in range(radar1.PTM_len):
            srx[:, p, :] = np.matmul(np.transpose(rxmat[:, p][np.newaxis]),antenna_matrix[np.newaxis])
    srx_volts = np.transpose(srx, (0, 2, 1))
    return srx_volts

def get_max_y(y_res):
    y_antenna = np.transpose(y_res[0])
    win_hamm_col = np.hamming(181)
    awin_col = np.sum(win_hamm_col)
    power_music = 10*np.log10(np.abs(y_antenna[511:])/awin_col) + np.float32(18.706)

    index_max = np.unravel_index(np.argmax(power_music, axis=None), power_music.shape)
    max_power = power_music[index_max]
    power_music[index_max] = power_music[index_max] + 5
    return max_power, power_music

In [4]:
def write_hardware_target(array, snr_db):
    golden_data_t1 = open("./Results/P200_Vd2_snr_"+str(snr_db)+".dat", "a")
    array_line = ' '.join(map(str, array))
    golden_data_t1.write(array_line + "\n")
    golden_data_t1.close()

In [5]:
radar1_in = radar1()
doppler_axis = np.arange(radar1_in.im_fdmin, radar1_in.im_fdmax + radar1_in.dfd, radar1_in.dfd)
velocity_axis = doppler_axis * radar1_in.Lambda/2
delay_axis = np.arange(-radar1_in.L + 1, radar1_in.L)
range_axis = ((radar1_in.c * radar1_in.ts) / 2) * delay_axis
theta_axis = np.arange(-90, 91, 1)
# Load XMAT
tx = sio.loadmat("tx_freq_domain.mat")["xmat"]
xmat = np.transpose(np.transpose(tx)[:nop])
xmat1=xmat*np.squeeze(np.tile(np.exp(1j * np.pi * np.arange(4)/ 2),(1,int(xmat.shape[0] / 4)))).reshape(-1,1)
xmat_padded1 = np.zeros((1024, nop),np.complex64)
xmat_padded1[0:radar1_in.L,:] = xmat1
xmat_padded = np.zeros((Nu_elements, 1024, nop), dtype=np.complex64)
for i in range(Nu_elements):
    xmat_padded[i] = xmat_padded1
xmat_padded = np.transpose(xmat_padded, (2, 0, 1))
xmat_fft_conj_tran = np.zeros(shape=np.shape(xmat_padded), dtype=np.complex64)
for i in range(radar1_in.PTM_len):
    xmat_fft_conj_tran[i] = np.conj(np.fft.fft(xmat_padded[i]))
n_frame = 1


In [6]:
 # Adding doppler delay 
def add_doppler_delay(rxmat_fft, doppler_delay):
    return np.transpose([a*b for a,b in zip(rxmat_fft, doppler_delay)])

def select_theta(avg_pre):
    return np.arange(-90, 91, avg_pre)
    
    # Method for multiplication for Diagonal elements of matrix
def diag_mat_mul(rxmat_dd, xmat_conj):
    xmat_conj_trans = np.transpose(xmat_conj)
    diag_res2 = np.sum(np.multiply(rxmat_dd, xmat_conj_trans), axis = 1)
    return diag_res2
    
def dbf(rxmat_fft, azi_delay):
    rxmat_dd=np.transpose([a*b for a,b in zip(rxmat_fft, azi_delay)])
    diag_res2_sum=np.sum(rxmat_dd,axis=1)
    return diag_res2_sum
    
def matchedfiltering_angle_v4(rxmat_fft, xmat_fft_conj_tran,k,r_in,IDFT,avg_pre=1):
    if(r_in>=1024):
        r_in=1023
    theta = select_theta(avg_pre)
    y = np.zeros(shape=(radar1_in.PTM_len-1,), dtype=np.complex64)
    temp = np.zeros(shape=(1, 2 * radar1_in.L), dtype=np.complex64)
    for i in range(radar1_in.PTM_len-1):
        theta_k = (2 * np.pi * np.sin(np.pi * theta[k] / 180)) / radar1_in.Lambda
        azi_delay = np.exp(-1j * theta_k * antenna_param.spacing)
        out=dbf(rxmat_fft[i], azi_delay)
        out1=np.multiply(out,xmat_fft_conj_tran[i+1])
        y[i]=np.matmul(IDFT[r_in,:][np.newaxis],np.transpose(out1[np.newaxis]))/1024 
    return y

def matchedfiltering2D(total_srx_noise, xmat_fft_conj_tran, avg_pre=1):
    rxmat_fft = np.zeros(shape=np.shape(total_srx_noise), dtype=np.complex64)
    rxmat_fft = np.fft.fft(total_srx_noise)

    theta = select_theta(avg_pre)
    y = np.zeros(shape=( len(theta), 2 * radar1_in.L), dtype=np.complex64)
    for k in range(len(theta)):
        theta_k = (2 * np.pi * np.sin(np.pi * theta[k] / 180)) / radar1_in.Lambda
        azi_delay = np.exp(-1j * theta_k * antenna_param.spacing)
        out=dbf(rxmat_fft, azi_delay)
        out1=np.multiply(out,xmat_fft_conj_tran)
        y[k][:] = np.fft.ifftshift(np.fft.ifft(out1))
    return y, theta

In [7]:
def get_points_2d(N, Rmax):
    D = 2
    x = np.random.randn(N, D)
    Rmin = 1
    Rad = Rmax
    
    y = np.zeros((N, D))
    r = np.zeros(N)
    
    for n in range(N):
        dr = np.linalg.norm(x[n, :])
        U = np.random.random() ** (1 / D)
        y[n, :] = U * Rad * x[n, :] / dr
        r[n] = np.linalg.norm(y[n, :])
        
        if r[n] < Rmin:
            y[n, :] = np.full(D, np.finfo(float).tiny)
    
    x_coordinate = y[:, 0]
    y_coordinate = y[:, 1]
    
    return x_coordinate, y_coordinate

In [8]:
# Single Target SNR Performance
Nu_elements = 32 # Variable Number of Antenna
np.random.seed(1)
radar_pos = np.array([0, 0])
snr_db = 10
N = 200
Rmax = 40
x_coordinate, y_coordinate = get_points_2d(N, Rmax)

range1 = np.round(np.sqrt((radar_pos[0] - x_coordinate) ** 2 + (radar_pos[1] - y_coordinate) ** 2))
theta1 = np.round(np.rad2deg(np.arctan2(y_coordinate, x_coordinate)))
range1[range1 == 0] = 1
x_coordinate, y_coordinate = get_points_2d(N, Rmax)
theta1[(np.where(np.logical_and(theta1>=90, theta1<=180)))]=theta1[(np.where(np.logical_and(theta1>=90, theta1<=180)))]-180
theta1[(np.where(np.logical_and(theta1>=-180, theta1<=-90)))]=theta1[(np.where(np.logical_and(theta1>=-180, theta1<=-90)))]+180
vel1 = np.random.randint(-29, 17, size=N)
rcs1 = np.round(np.random.exponential(2, size=N))
vel2 = vel1 + 12
rcs2 = np.round(np.random.exponential(2, size=N))
theta1[theta1 > 85] = 85
theta1[theta1 < -85] = -85
rcs1[rcs1 == 0] = 1
rcs2[rcs2 == 0] = 1
r = np.transpose(np.vstack((range1)))
r1=range1
o1=theta1
v1=vel1
v2=vel2
sigma1=rcs1
sigma2=rcs2

In [9]:
ol = Overlay("esprit.bit")
data_channel = ol.axi_dma_0
import time
pltime = 0

pstime = 0

# Function to perfrom Autocorrelation
def autocorrelation_func(rec_sig):
    conj_sig = np.conj(rec_sig)
    autocorrelation_matrix = []
    for i in range(len(conj_sig)):
        dat1 = conj_sig[i] * rec_sig
        autocorrelation_matrix.append(dat1.astype(np.complex64))
    return np.array(autocorrelation_matrix,dtype = np.complex64)

def Phi_cal(A, B,det_A,k):
    
    a11, a12 = A[0, 0], A[0, 1]
    a21, a22 = A[1, 0], A[1, 1]
    
    b11, b12 = B[0, 0], B[0, 1]
    b21, b22 = B[1, 0], B[1, 1]

    phi = np.zeros((k, k), dtype=np.complex64)
    
    # Calculate phi matrix elements
    phi[0,0] = (a22 * b11 - a12 * b21)
    phi[0,1] = (a22 * b12 - a12 * b22)
    phi[1,0] = (-a21 * b11 + a11 * b21)
    phi[1,1] = (a11 * b22 - a21 * b12)

    phi /=det_A
    return phi

def eign_cal(phi_mat):
    
    trace = phi_mat[0,0] + phi_mat[1,1]
    determinant = phi_mat[0, 0] * phi_mat[1, 1] - phi_mat[0, 1] * phi_mat[1, 0]
    discriminant = trace**2 - np.complex64(4.0) * determinant
    lambda1 = (trace + np.sqrt(discriminant, dtype=np.complex64)) / np.complex64(2.0)
    lambda2 = (trace - np.sqrt(discriminant, dtype=np.complex64)) / np.complex64(2.0)
    return np.array([lambda1, lambda2], dtype=np.complex64)


def check_and_replace(arr):
    
    if -30 <= arr[0] <= 30 and -30 <= arr[1] <= 30:
        return arr
    elif -30 <= arr[0] <= 30:
        arr[1] = arr[0]
    elif -30 <= arr[1] <= 30:
        arr[0] = arr[1]
    else:
        arr = [0, 0]
    return arr

# Function to generate rotation matrix
def givensrotation(a, b):
    z = a*a + b*b
    r = np.sqrt(z.real**2 + z.imag**2)
    theta = np.arctan2(z.imag, z.real)
    hypo = np.sqrt(r) * (np.cos(theta / 2) + 1j * np.sin(theta / 2))
    if hypo == 0:
        return 1, 0
    else:
        cos1 = a/ hypo
        sin1 = -b / hypo
        return cos1, sin1
    
# QR using Givens Rotation
def qr_givens(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m,dtype=np.complex64)
    for i in range(0, n - 1):
        for j in range(i + 1, m):
            cos, sin = givensrotation(R[i, i], R[j, i])
            R[i], R[j] = (R[i]*cos) + (R[j]*(-sin)), (R[i]*sin) + (R[j] * cos)
            Q[i], Q[j] = (Q[i]*cos) + (Q[j]*(-sin)), (Q[i]*sin) + (Q[j] * cos)
    return np.transpose(np.conj(Q)), R

def ESPRIT_PS(rec_signal):
    rec_signal = rec_signal.astype(np.complex64)
    N = len(rec_signal)
    l = N//2
    k = 2
    subarr = N + 1 - l
    autocorrelation_matrix = np.zeros((l, l), dtype=np.complex64)
    for i in range(subarr):
        sub1 = rec_signal[i:(l + i)]
        sub2 = autocorrelation_func(np.array(sub1))
        autocorrelation_matrix += sub2
    autocorrelation_matrix /= subarr
    eign_vec, eign_val = qr_givens(autocorrelation_matrix)
    S = eign_vec[:, :k]

    phi_mat = np.zeros((k, k), dtype=np.complex64)
    iter1 = 0
    for i in range(0,l-1,1):
        subA1 = S[i:i+k, :]
        subB1 = S[i+1:i+k+1,:]
        if subA1.shape[0] == 2 and subB1.shape[0] == 2:
            det_A = subA1[0, 0] * subA1[1, 1] - subA1[0, 1] * subA1[1, 0]
            if det_A == 0:
                phi_mat = phi_mat
            else:
                phi_mat += Phi_cal(subA1, subB1,det_A,k)
                iter1 +=1

    phi_mat /= iter1
    phi = eign_cal(phi_mat)
    omega_estimates = np.angle(phi.astype(np.complex64)).astype(np.float32) / np.float32(0.005)
    omega_estimates = check_and_replace(omega_estimates)
    omega_estimates = np.sort(omega_estimates)[::-1]
    return omega_estimates

def music1(rec_signal1):
    input_buffer = allocate((200,),np.csingle)
    output_buffer = allocate((2,),np.single)
    
    send_channel = data_channel.sendchannel
    recv_channel = data_channel.recvchannel
    
    np.copyto(input_buffer,rec_signal1)

    send_channel.transfer(input_buffer)
    recv_channel.transfer(output_buffer)
    send_channel.wait()
    recv_channel.wait()

    return output_buffer


def ESPRIT_PL(rec_signal1):
    p_music = music1(rec_signal1)
    
    while(np.isnan(p_music[0])):
        rec_signal1 = rec_signal1 * np.float32(3.1622776602)/2
        p_music = music1(rec_signal1)
        print("nan detected")
        
    omega_estimates = check_and_replace(p_music)
    omega_estimates = np.sort(omega_estimates)[::-1]
    
    return omega_estimates[0], omega_estimates[1]


def ESPRIT_F(rec_signal1):
    
    start_time = time.time()
    vx1, vx2 = ESPRIT_PS(rec_signal1)
    finish_time = time.time()
    pstime = finish_time - start_time
    print(f"PS Time = {pstime}")
    
    start_time = time.time()
    vx3, vx4 = ESPRIT_PL(rec_signal1)
    finish_time = time.time()
    pltime = finish_time - start_time
    print(f"PL Time = {pltime}") 
    
    acc_fac = pstime/pltime
    
    print(f"Acceleration factor = {acc_fac}")
    
    
    return vx1, vx2, vx3, vx4, pstime, pltime, acc_fac

In [10]:
p_acc_fac = []
pstime_all = []
pltime_all = []
acc_fac_avg = 0
for j in range(40):
    print("Round: " + str(j))
    N=5
    snr_db = 10
    signal_power = np.zeros(N)
    srx_volts = np.zeros((N, radar1_in.PTM_len, Nu_elements, 2 * radar1_in.L), dtype=np.complex64)
    srx_noise = np.zeros((N, radar1_in.PTM_len, Nu_elements, 2 * radar1_in.L), dtype=np.complex64)
    rxmat_fft = np.zeros(shape=np.shape(srx_noise[1:,:,:]), dtype=np.complex64)
    n1 = 1024
    w1 = np.exp(-1j * 2 * np.pi / n1)
    I, J = np.meshgrid(np.arange(1, n1 + 1), np.arange(1, n1 + 1))
    DFT1 = w1**((I - 1) * (J - 1))
    IDFT = np.transpose(np.conj(DFT1))
    mean_signal_power=(10*32*1*(np.float32(0.005)**2)*2)/(((4*np.pi)**3)*(40**4))
    for snr_db in [-15,-10,-5,0,5,10,15,20,25,30,35,40]:
        print("SNR:", snr_db)
        snr_linear = 10 ** (snr_db/10)
        noise_power = mean_signal_power/snr_linear
        for i in range(j*N,(j+1)*N):
            range_t=r1[i];
            velocity1=v1[i];
            velocity2=v2[i];
            rcs_amp1=sigma1[i];
            rcs_amp2=sigma2[i];
            DOA_angle=o1[i];
            t1 = target(range_t, velocity1, rcs_amp1)
            t2 = target(range_t, velocity2, rcs_amp2) 
            antenna_param = antenna(radar1_in.Lambda, Nu_elements, DOA_angle)
            rxmat_t1 = get_received_signal(t1, 
                                            radar1_in, 
                                            xmat1, 
                                            radar1_in.fs, 
                                            radar1_in.fc, 
                                            antenna_param, 
                                            n_frame)

            rxmat_t2 = get_received_signal(t2, 
                                            radar1_in, 
                                            xmat1, 
                                            radar1_in.fs, 
                                            radar1_in.fc, 
                                            antenna_param, 
                                            n_frame)
            rxmat_t12 = rxmat_t1 + rxmat_t2
            srx_volts = np.transpose(rxmat_t12, (2, 1, 0))
            noise = np.sqrt(noise_power) * (np.random.randn(srx_volts.shape[0],srx_volts.shape[1],srx_volts.shape[2]) + (1j * np.random.randn(srx_volts.shape[0],srx_volts.shape[1],srx_volts.shape[2])))
            srx_noise = srx_volts + noise
            rxmat_fft = np.fft.fft(srx_noise[1:,:,:])
            
            y_pack1 = np.zeros(shape=(radar1_in.PTM_len,), dtype=np.complex64)
            y_p1 = np.zeros(shape=(radar1_in.PTM_len-1,), dtype=np.complex64)
            
            y_res, theta = matchedfiltering2D(srx_noise[0,:,:], xmat_fft_conj_tran[0,0,:], avg_pre=angle_prec) 
            [a1, b1] = np.unravel_index(np.argmax(np.abs(y_res)), y_res.shape)
            mf_t1=y_res[a1,b1]
            if(b1<=512):
                b1_new=b1+512
            else:
                b1_new=b1-512
            y_p1 = matchedfiltering_angle_v4(rxmat_fft, xmat_fft_conj_tran[:,0,:],a1,b1_new,IDFT)
            y_pack1[1:radar1_in.PTM_len]=y_p1
            y_pack1[0]=mf_t1
            music_in1=y_pack1
            music_in = music_in1.flatten().astype(np.complex64)
            vx1, vx2, vx3, vx4, pstime, pltime, af = ESPRIT_F(music_in)
            acc_fac_avg += af
            p_acc_fac.append(af)
            pstime_all.append(pstime)
            pltime_all.append(pltime)

            
            hw = np.array([velocity2,velocity1,vx1, vx2, vx3, vx4])
            
            print(f"Golden: {velocity2}, {velocity1}, PS: {vx1}, {vx2}, PL: {vx1}, {vx2}")

            write_hardware_target(hw, snr_db)
            
        acc_fac_avg /= 5
        
        print(f"Average acceleration factor over 5 runs for SNR = {snr_db} is {acc_fac_avg}")
p_acc_fac = np.array(p_acc_fac)
pstime_all = np.array(pstime_all)
pltime_all = np.array(pltime_all)

Round: 0
SNR: -15
PS Time = 2.3051881790161133
PL Time = 0.0732278823852539
Acceleration factor = 31.479650973497428
Golden: 28, 16, PS: 28.255800247192383, 28.255800247192383, PL: 28.255800247192383, 28.255800247192383
PS Time = 2.291912794113159
PL Time = 0.07274508476257324
Acceleration factor = 31.50608459105583
Golden: 17, 5, PS: 17.539058685302734, 3.6734373569488525, PL: 17.539058685302734, 3.6734373569488525
PS Time = 2.3053274154663086
PL Time = 0.0726935863494873
Acceleration factor = 31.71294100669402
Golden: 14, 2, PS: 15.02939510345459, 15.02939510345459, PL: 15.02939510345459, 15.02939510345459
PS Time = 2.301496982574463
PL Time = 0.07632064819335938
Acceleration factor = 30.15562678062678
Golden: 17, 5, PS: 18.151805877685547, 18.151805877685547, PL: 18.151805877685547, 18.151805877685547
PS Time = 2.3150556087493896
PL Time = 0.07939791679382324
Acceleration factor = 29.157636651362235
Golden: -10, -22, PS: -9.520394325256348, -9.520394325256348, PL: -9.520394325256348

In [11]:
np.shape(p_acc_fac)

(2400,)

In [12]:
p_acc_fac

array([31.47965097, 31.50608459, 31.71294101, ..., 30.70976484,
       30.55166305, 30.45366647])

In [13]:
np.dtype(vx1)

dtype('float32')

In [14]:
vx2

-4.022733

In [15]:
np.shape(p_acc_fac)

(2400,)

In [16]:
p_acc_fac

array([31.47965097, 31.50608459, 31.71294101, ..., 30.70976484,
       30.55166305, 30.45366647])

In [2]:
np.mean(p_acc_fac)

NameError: name 'np' is not defined